# Poor man's adaptive sweep

Using Adaptive sweeps instead of regular sweeps can save a lot of time.
Currently, there is no deep integration in `pipefunc` to do adaptive sweeps.
However, we can still do a poor man's version of them.


In the future the idea is to allow a syntax like this:

```python
pipeline.map(inputs={'a': Bound(0, 1), 'b': Bound(0, 1), c=[0, 1, 2]})
```

This will turn into a 2D adaptive sweep over `a` and `b` and do that for each value of `c`.

For now, we can do the following "hack".

## Setting the stage

Let's set the stage by setting up a simple pipeline with a reduction operation.

In [ ]:
from pipefunc import pipefunc, Pipeline


@pipefunc(output_name="y", mapspec="x[i] -> y[i]")
def double_it(x: int, c: int) -> int:
    return 2 * x + c


@pipefunc(output_name="sum_")
def take_sum(y: list[int], d: int) -> float:
    return sum(y) / d


pipeline = Pipeline([double_it, take_sum])

inputs = {"x": [0, 1, 2, 3], "c": 1, "d": 2}
run_folder = "my_run_folder"
results = pipeline.map(inputs, run_folder=run_folder)
print(results["y"].output.tolist())
assert results["y"].output.tolist() == [1, 3, 5, 7]
assert results["sum_"].output == 8.0

This pipeline returns a single number, which is the sum of the inputs.

However, often we want to run a pipeline for a range of inputs, on e.g., a 2D grid on `c` and `d`.

In [ ]:
pipeline2d = pipeline.copy()
pipeline2d.add_mapspec_axis("c", axis="j")
pipeline2d.add_mapspec_axis("d", axis="k")

Now let's run this on a 2D grid of `c` and `d`:

In [ ]:
import numpy as np

inputs = {"x": [0, 1, 2, 3], "c": np.linspace(0, 100, 20), "d": np.linspace(-1, 1, 20)}
run_folder = "my_run_folder"
results = pipeline2d.map(inputs, run_folder=run_folder)

We can load the results into an xarray dataset and plot them.

In [ ]:
from pipefunc.map import load_xarray_dataset

ds = load_xarray_dataset(run_folder=run_folder)
ds.sum_.astype(float).plot(x="c", y="d")

## Doing it with an `adaptive.Learner2D`

In [ ]:
import adaptive

adaptive.notebook_extension()

We redefine the `pipeline` with the single reduction operation.

In [ ]:
from pipefunc import pipefunc, Pipeline


@pipefunc(output_name="y", mapspec="x[i] -> y[i]")
def double_it(x: int, c: int) -> int:
    return 2 * x + c


@pipefunc(output_name="sum_")
def take_sum(y: list[int], d: int) -> float:
    return sum(y) / d


pipeline = Pipeline([double_it, take_sum])

In [ ]:
from pipefunc.map.adaptive import to_adaptive_learner

run_folder_template = "adaptive_1d/run_folder_{}"
learner1d = to_adaptive_learner(
    inputs={"x": [0, 1, 2, 3], "d": 1},
    adaptive_dimensions={"c": (0, 100)},
    adaptive_output="sum_",
    run_folder_template=run_folder_template,
)
adaptive.runner.simple(learner1d, npoints_goal=10)

We can now inspect the results of the `adaptive_output` in the learner

In [ ]:
learner1d.to_numpy()

In [ ]:
learner1d.plot()

Or inspect all the underlying data

In [ ]:
from pathlib import Path

from pipefunc.map import load_xarray_dataset

all_folders = Path(run_folder_template).parent.glob("*")
all_folders = sorted(all_folders)
datasets = [load_xarray_dataset(run_folder=folder) for folder in all_folders]

In [ ]:
datasets[0]

In [ ]:
run_folder_template = "adaptive_2d/run_folder_{}"
learner2d = to_adaptive_learner(
    inputs={"x": [0, 1, 2, 3]},
    adaptive_dimensions={"c": (0, 100), "d": (-1, 1)},
    adaptive_output="sum_",
    run_folder_template=run_folder_template,
)
adaptive.runner.simple(learner2d, npoints_goal=10)